<a href="https://colab.research.google.com/github/theharshithh/onfinace-assignment/blob/main/parent%2Bchild_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [5]:
!pip -q --quiet install langchain openai tiktoken chromadb langchain_openai cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.7 MB/s eta 0:00:00


In [3]:
!pip install --quiet langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 1.7 MB/s eta 0:00:00


In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

# from langchain_cohere import CohereEmbeddings
import cohere
from google.colab import userdata
cohere_api_key = userdata.get('COHERE_API_KEY')
openai_api_key = userdata.get('OPENAI_API_KEY')


co = cohere.Client(cohere_api_key)


In [ ]:
file_loader = PyPDFLoader('dataset/sample_data.pdf')
docs = file_loader.load()
docs

loading data to /dataset dir

In [ ]:
!pip --quiet install pypdf

creating chunks for the `primary_data`

In [ ]:
primary_data = docs
primary_data


# primary_data = [doc.page_content for doc in docs]

In [129]:
persist_directory = 'db-trash'

#openai embeddings -> converts texts to vecs
embedding = OpenAIEmbeddings(openai_api_key = openai_api_key)
# embedding = OpenAIEmbeddings(openai_api_key = openai_api_key, model="text-embedding-3-small")


vectordb = Chroma.from_documents(documents=spilit_texts, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [133]:

# Use your trash data retriever to get the relevant documents
trash_retriever = vectordb.as_retriever(search_kwargs={"k":20}) #k: 20 for more context window to llm
trash_text = trash_retriever.get_relevant_documents('Get me the SEBI disclosure and the unnecessary disclaimers and terms and conditions')

trash_metadata = [doc.metadata for doc in trash_text]
main_data_filtered = [doc for doc in primary_data if doc.metadata not in trash_metadata]

main_data_filtered


[Document(page_content=" Automobiles  \n27 March 2023    3 \nSnapshot of Mar ’23 volume estimates                \n  YoY     MoM        \nCompany Sales  Mar -23 Mar -22 YoY (%)  \n chg Feb-23 MoM (%) \nchg FY23E  FY22  (%) \n chg \nMaruti Suzuki  1,64,812  1,70,395  -3.3 1,72,321  -4.4 19,60,905  16,52,653  18.7  \nLCVs  3,887  3,797  2.4 3,356  15.8  37,869  33,812  12.0  \nVans  11,901  9,221  29.1  11,352  4.8 1,31,097  1,08,345  21.0  \nMini Segment  20,020  15,491  29.2  21,875  -8.5 2,41,349  2,11,762  14.0  \nCompact (incl Dzire Tour)  74,673  84,761  -11.9  81,325  -8.2 8,95,245  7,26,443  23.2  \nMid-Size - CIAZ  972 1,834  -47.0  792 22.7  14,282  15,869  -10.0  \nUVs 36,931  28,795  28.3  36,414  1.4 3,95,421  3,18,046  24.3  \nDomestic  1,48,385  1,43,899  3.1 1,55,114  -4.3 17,15,264  14,14,277  21.3  \nExport  16,427  26,496  -38.0  17,207  -4.5 2,45,641  2,38,376  3.0 \nMahindra & Mahindra  94,804  84,406  12.3  84,592  12.1  10,99,710  8,20,299  34.1  \n   UV (incl. pic

In [128]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)

parents_vectordb = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings(openai_api_key=openai_api_key)
)

# memory to store the parent layer.
store = InMemoryStore()

In [111]:
full_vectordb = Chroma(
    collection_name="full_documents",
    embedding_function= OpenAIEmbeddings(openai_api_key = openai_api_key)
)

store = InMemoryStore()

full_doc_retriever = ParentDocumentRetriever(
    vectorstore = full_vectordb,
    docstore = store,
    child_splitter=child_splitter,
)

In [122]:
full_doc_retriever.add_documents(main_data_filtered, ids=None)

In [118]:
list(store.yield_keys())

['60fbce9a-7dc2-4305-80a2-1e98ed622fb0',
 'a34e1346-106c-48b5-9a96-9670922ac76d']

In [115]:
user_query = 'Based on the current 2 wheeler demand, who is the market leader?'

In [119]:
sub_docs = vectordb.similarity_search(user_query)
sub_docs[0].page_content

'Hero MotoCorp  4,34,165  4,50,170  -3.6 3,94,460  10.1  52,43,369  49,44,148  6.1 \nBajaj Auto  2,81,922  2,97,188  -5.1 2,80,226  0.6 39,16,077  43,08,433  -9.1 \n  Two -Wheelers  2,33,703  2,56,324  -8.8 2,35,356  -0.7 34,27,405  38,36,856  -10.7  \n  Three -Wheelers  48,219  40,864  18.0  44,870  7.5 4,88,672  4,71,577  3.6'

In [ ]:
retrived_docs = full_doc_retriever.get_relevant_documents(user_query)
retrived_docs

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k":8}) #k: 20 for more context window to llm
docs = retriever.get_relevant_documents(user_query)
docs

### LLM chain

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

compressor = CohereRerank(cohere_api_key = cohere_api_key, model = 'rerank-english-v3.0') #   top_n = 3 -> can limit the sources
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever,
)

compressed_docs = compression_retriever.get_relevant_documents(user_query)
compressed_docs

In [78]:
# create the chain to answer question
qa_chain_compressed = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=openai_api_key,model="gpt-3.5-turbo", temperature=1.2 ), chain_type="stuff", retriever=compression_retriever, return_source_documents=True)

In [114]:
# create the chain to answer question
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(openai_api_key=openai_api_key,model="gpt-3.5-turbo", temperature=1.2 ), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [80]:
def process_llm_response(llm_response):
    print("Decision:")
    print(llm_response['result'])

    print("\n Source Documents:")
    for source in llm_response["source_documents"]:
        print("- " + source.metadata['source'])

    print("\nRelevant Data:")
    for source in llm_response["source_documents"]:
        print(source.page_content)

compressed qa chain for efficent retrival

In [ ]:
behaviour = '''Think like an expert finance consultant from a top tier firm. Do all the research and analysis for this document.'''
prompt_template = 'Keep the answer crisp. Give me as much numbers as you can. Every Statement of yours should be backed with data'
stop_prompt = f'Dont hallucinate, if u dont know the answer, tell you dont know'
query = f"Here is the user query : {user_query}. Remember this: {behaviour}. Give me the answers like this: {prompt_template}. Keep this in mind - {stop_prompt}"
llm_response = qa_chain_compressed(query)
process_llm_response(llm_response)

best version -> heavy credits burning -> deep pockets required

In [ ]:
llm_response = qa_chain(query)
process_llm_response(llm_response)

In [ ]:
qa_chain